In [3]:
import threading
import time

# Paczka Threading umożliwia tworzenie wątków na dwa sposoby. Pierwszy to stworzenie klasy, w której w metodzie run bez argumentów, która zawiera logike wątku.

In [11]:
class Task(threading.Thread):
    def run(self):
        print("Task started")
        time.sleep(2)
        print("Task done")
        
Task().start()
print("Next line")

Task started
Next line
Task done


# Do czekania w wątku na jego zakończenie służy metoda join() klasy Thread

In [12]:
class Task(threading.Thread):
    def run(self):
        print("Task started")
        
        time.sleep(2)
        print("Task done")
        
watek=Task()
watek.start()
watek.join()#<-------------- TUTAJ
print("Next line")

Task started
Task done
Next line


# Wątek daemon to wątek, który zostanie zatrzymany bez ostrzeżenia przez interpreter
## dopisujemy wtedy : watek.daemon=True

# Obsługa wątków przy pomocy funkcji zwiększa elastyczność korzystania z nich i jest wskazana np. ze względu na łątwiejsze testowanie

In [15]:
#definiujemy funkcje
def zadanie():
    time.sleep(2)
    print("Task done")
    
#wywołujemy wątek, przywołując moduł Thread
threading.Thread(target=zadanie, args=(), kwargs=None, daemon=False).start()

Task done


# Bariera to miejsce w funkcji, która będzie wykonywana przez kilka wątków, która zatrzymuje wątki, dopóki wszystkie (określone przy deklarowaniu bariery) nie dotrą do tego miejsca.

In [30]:
# wyscigi psow
import random
dogNames=["Nikos", "Molly", "Alfa", "Nora"]

runStages=["Ready...", "Started!", "Finished!!"]

def wait():
    time.sleep(random.randint(1,3))
    
def onStart():
    print("-----Race started!------")

def run(name, barrier):
    
    for stage in runStages:
        wait()
        print("{} {}".format(name, stage))
        if stage=="Ready...":
            barrier.wait()
            
            
barieraStartu=threading.Barrier(len(dogNames), action=onStart)
dogs=[threading.Thread(target=run, args=(x, barieraStartu)) for x in dogNames]

for dog in dogs:
    dog.start()
    

for dog in dogs:
    dog.join()

    

Molly Ready...
Nora Ready...
Alfa Ready...
Nikos Ready...
-----Race started!------
Nora Started!
Nikos Started!
Molly Started!Alfa Started!
Nikos Finished!!

Nora Finished!!
Alfa Finished!!Molly Finished!!



# Piszemy wątek reagujący na klawisz enter, wykorzystując obiekt Event z Threading

In [33]:
import threading

keyPressed=threading.Event()
finished=threading.Event()
def on_key_press():
    while not finished.is_set():
        if keyPressed.wait(0.1):
            print('key pressed000')
            keyPressed.clear()
        
        print("done")
        
for _ in range(4):
    input()
    keyPressed.set()

    
threading.Thread(target=on_key_press).start()

finished.set()






key pressed000
done


# -----synchronizacja dostepu do zasobow-----

1. Ukrywanie danych w włąsnej pamięci wątku. Każdy wątek będzie miał do dyspozycji własną niepowtarzalną kopię tej samej zmiennej. Musimy ją zadeklarować jako 
### zmienna=threading.local()

powyższego nie należy stosować w puli wątków, co naraża nas na błędy i wycieki pamięci. Nie mamy gwarancji że wtedy to samo zadanie będzie wykonywane przez ten sam wątek 

2. Blokada - synchronizacja dostępu do współdzielonego zasobu, która blokuje dany fragment kodu tylko dla jednego wątku na raz

In [35]:
nasza_blokada=threading.Lock()
with nasza blokada:
    pass
    #tutaj nasz fragment kodu, który chcemy sobie zablokowac

SyntaxError: expected ':' (4044826707.py, line 2)

# Funkcja print() nie jest bezpieczna wątkowo. Chcąc jednocześnie pisać z wielu wątków używamy modułu logging

In [36]:
import logging

In [37]:
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [1]:
import queue

In [5]:
kolejka=queue.Queue(maxsize=3)

In [7]:
for _ in range(3): 
    kolejka.put("jeden")

In [9]:
kolejka.task_done()

# Real Python - wstęp do wielowątkowości

In [17]:
import threading
import time
import random


def thread_function(thread_name):
    print('{} 1'.format(thread_name))
    time.sleep(random.random())
    print('{} 2'.format(thread_name))
    time.sleep(random.random())
    print('{} 3'.format(thread_name))
    time.sleep(random.random())
    print('{} 4'.format(thread_name))
    time.sleep(random.random())
    
thread_names= [
    'Pierwszy',
    'Drugi',
    'Trzeci'
]
    
for thread in thread_names:
    threading.Thread(target=thread_function, args=(thread,),daemon=True).start()
    
    
print("Koniec programu")


Pierwszy 1
Drugi 1
Trzeci 1
Koniec programu
Pierwszy 2
Trzeci 2
Drugi 2
Pierwszy 3
Trzeci 3
Pierwszy 4
Drugi 3
Trzeci 4
Drugi 4


In [14]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,), daemon=True)
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    #x.join()
    logging.info("Main    : all done")

22:09:24: Main    : before creating thread
22:09:24: Main    : before running thread
22:09:24: Thread 1: starting
22:09:24: Main    : wait for the thread to finish
22:09:24: Main    : all done
22:09:26: Thread 1: finishing


# łatwa implementacja uruchamiania wiecej niz jednego wątka poniżej

In [21]:

import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as Th:
    Th.map(thread_function, thread_names)
    
print("koniec")

Pierwszy 1
Drugi 1
Trzeci 1
Trzeci 2
Drugi 2
Trzeci 3
Drugi 3
Drugi 4
Pierwszy 2
Trzeci 4
Pierwszy 3
Pierwszy 4
koniec


# Thread.Lock() --> blokada, której dwie głównie wywoływane metody to .acquire() i .release(). Minusem jest to, że jeżeli acquire blokuje wątek. RLock() nie blokuje

# Eventy i Queue

Plusem Eventów jest to, że wątki nie muszą przerywać swojego działania, mogą tylko co jakiś czas sprawdzać stan eventu.

Queue jest bezpieczna przy pracy z wieloma wątkami, metody get i set zawierają mechanizmy blokad

# TImery z paczki threading pozwalają na uruchomienie funkcjo po określonym czasie

In [22]:
t=threading.Timer(5, thread_function)